In [1]:
cd ../

In [2]:
from fairrecs.preprocessor import preprocess_yow
from fairrecs.solver import Solver

In [3]:
dataset = preprocess_yow('datasets/yow_userstudy_raw.csv')
dataset.head()

,EventOnScroll,MSecForDownArrow,TimeOnVScroll,relevant,NumOfPageUp,TimeOnMouse,NumOfPageDown,ClickOnWindow,log_id,serverTimeVisit,...,readability,TimeOnHScroll,novelty,user_like,DOC_ID,TimeOnPage,TimeVisit,user_id,NumOfDownArrow,authority
9780,0.0,0.0,0.0,0.638221,0,2156,0.0,0.0,5851,2004-05-02 18:40,...,1.0,0.0,2.0,3,295842,106781.0,2004-05-02 18:40,92,0.0,1.0
9781,0.0,0.0,0.0,0.376913,0,3934,0.0,0.0,5855,2004-05-02 18:42,...,1.0,0.0,4.0,4,282679,129141.0,2004-05-02 18:42,92,0.0,1.0
9782,0.0,0.0,0.0,0.380992,0,4280,0.0,1.0,5861,2004-05-02 18:45,...,1.0,0.0,2.0,3,295838,107500.0,2004-05-02 18:45,92,0.0,1.0
9784,0.0,0.0,0.0,0.473998,0,3339,0.0,0.0,5868,2004-05-02 18:50,...,1.0,0.0,2.0,4,281594,147469.0,2004-05-02 18:50,92,0.0,1.0
9785,0.0,0.0,0.0,0.552228,0,4561,0.0,1.0,5872,2004-05-02 18:53,...,1.0,0.0,2.0,4,294299,178390.0,2004-05-02 18:53,92,0.0,1.0


In [4]:
solver = Solver()
P = solver.solve(dataset['relevant'].to_numpy())
print(P)

[[ 1.08654282e-05  1.08684628e-05 -2.08788834e-05 ...  2.40869253e-06
  -4.88167047e-06  4.48163889e-06]
 [ 3.54810020e-06  3.44626498e-06 -2.83382844e-05 ...  2.26163981e-01
   2.47503691e-01  2.46392959e-01]
 [ 2.93590096e-05  2.92588116e-05 -2.52515691e-06 ...  2.15299839e-01
   2.11187435e-01  1.85671612e-01]
 ...
 [-7.15151134e-06 -7.09446591e-06 -3.88226514e-05 ... -1.54902106e-05
  -2.27803589e-05 -1.34168435e-05]
 [ 3.66700776e-06  3.56331272e-06 -2.82218964e-05 ...  2.27514544e-01
   2.77766820e-01  3.04379278e-01]
 [ 1.29903034e-05  1.30633241e-05 -1.86591941e-05 ...  4.68651681e-06
  -2.60356788e-06  6.76000835e-06]]
